Big trajectory with GCS

In [ ]:
import numpy as np
from functools import partial

from pydrake.all import (
    AddDefaultVisualization,
    DiscreteContactApproximation,
    PidController,
    RobotDiagramBuilder,
    Simulator,
    StartMeshcat,
    AddMultibodyPlantSceneGraph,
    AddUnitQuaternionConstraintOnPlant,
    AutoDiffXd,
    DiagramBuilder,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    JacobianWrtVariable,
    JointIndex,
    MathematicalProgram,
    MeshcatVisualizer,
    OrientationConstraint,
    Parser,
    PiecewisePolynomial,
    PositionConstraint,
    RotationMatrix,
    SnoptSolver,
    Solve,
    eq,
    namedview, 
    GraphOfConvexSets, 
    GraphOfConvexSetsOptions, 
    HPolyhedron, 
    Point
)
from pydrake.solvers import ClarabelSolver
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from underactuated import ConfigureParser, running_as_notebook
from underactuated.multibody import MakePidStateProjectionMatrix    


In [ ]:
#meshcat + create prog
meshcat = StartMeshcat()

In [ ]:
class Spot:
    """stores state of spot"""
    def __init__(self):
        self.com = None
        self.comdot = None
        self.comdotdot = None
        self.context = None
    
    def set_states(self, state_dict):
        self.com = state_dict['com']
        self.comdot = state_dict['comdot']
        self.comddot = state_dict['comddot']
        self.context = state_dict['context']


In [ ]:
class Obstacle:
    """obstacle w/ location and radius"""
    def __init__(self, x, y, rad):
        self.pos = (x, y)
        self.rad = rad
    
    def render(self, length = 10, m = 5, name=None, write_dir='./sdfs/'): #TODO: figure out file location
        """func to render in meshcat"""
        ixx = m*length**2/12 + m*self.rad**2/4
        iyy = ixx
        izz = m*self.rad**2/4
        half_length = length/2
        
        with open('obstacle.sdf', 'r') as c:
            sdf = c.read()
        sdf = sdf.replace('REPLACE_MASS', str(m)).replace('REPLACE_IXX', str(ixx)).replace('REPLACE_IYY', str(iyy)).replace('REPLACE_IZZ', str(izz)).replace('REPLACE_RADIUS', str(radius)).replace('REPLACE_LENGTH', str(length)).replace('REPLACE_HALF_LENGTH')

        if name is None:
            name = 'cylinder_' + str(len(os.listdir(write_dir))) + '.sdf'
        with open(write_dir+name, 'w') as s:
            s.write(sdf)
    
    

In [ ]:
class Environment:
    """environment w/ multiple obstacles"""
    def __init__(self, goal, N=50):
        self.obstacles = []
        self.spot_rad = 0.8 #implement radius from spot CoM to be away from obstacle
        self.N = N #num_timesteps
        self.goal = goal
        
    def add_obstacle(self, obs):
        """adds obstacle"""
        self.obstacles.append(obs)
    
    def _euclidean_dist(self, pos1, pos2):
        x1, y1 = pos1 
        x2, y2 = pos2
        return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

    def distance_from(self, point):
        """calculates distance of given point from nearest obstacle"""
        if not self.obstacles:
            return np.inf
        
        distances = [self._euclidean_dist(obs.pos, point) for obs in self.obstacles]
        return min(distances)  

    

In [ ]:
#visualization helpers

def plot_circle(center, radius, *args, **kwargs):
    """taken from orbital transfer notebook"""
    # discretize angle
    angle = np.linspace(0, 2 * np.pi)

    # plot circle
    plt.plot(
        center[0] + radius * np.cos(angle),
        center[1] + radius * np.sin(angle),
        *args,
        **kwargs
    )

def plot_env(env):
    for obs in env.obstacles:
        plt.scatter(*obs.pos, s=10, c='red')
        
        plot_circle(obs.pos, obs.rad, color = 'red', linestyle='--', linewidth = 0.5)
        plot_circle(obs.pos, obs.rad + env.spot_rad, color='blue', linestyle='--', linewidth = 0.5)
    plt.scatter(*env.goal, s=50, c='purple')
    plt.scatter(0, 0, s=50, c='purple')
    plt.grid(True)
    plt.gca().set_aspect("equal")
    ax = plt.gca()
    ax.set_xlim([0, 30])
    ax.set_ylim([-15, 15])

def plot_trajectory(trajectory):
    plt.plot(trajectory.T[0], trajectory.T[1], color="k", label="Rocket trajectory")
    plt.scatter(trajectory[0, 0], trajectory[0, 1], color="k")


In [ ]:
#add obstacles
obs1 = Obstacle(12, 3, 2)
obs2 = Obstacle(0, -10, 2)
obs3 = Obstacle(10, -15, 2)
obs4 = Obstacle(20, -8, 4)
obs5 = Obstacle(30, 10, 3)
env = Environment((30, 0), N=150)
env.add_obstacle(obs1)
env.add_obstacle(obs2)
env.add_obstacle(obs3)
env.add_obstacle(obs4)
env.add_obstacle(obs5)
plot_env(env)
plt.figure()

In [ ]:
class Position:
    def __init__(self, x_ll, y_ll):
        """lower left coordinates of a 5x5 tile"""
        self.x = x_ll
        self.y = y_ll
        self.center = (x_ll, y_ll)
        self.width = 3
        self.height = 3
        c2br = np.array([self.width, -self.height]) / 2
        c2tr = np.array([self.width, self.height]) / 2
        self.top_right = self.center + c2tr
        self.bottom_right = self.center + c2br
        self.top_left = self.center - c2br
        self.bottom_left = self.center - c2tr
        self.A = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
        self.b = np.concatenate([c2tr] * 2) + self.A.dot(self.center)
        self.name = f"({int(self.x-1.5)},{int(self.y-1.5)})"


In [ ]:
def get_loc_from_name(name):
    split_pos = name.split(',')
    x_i = int(split_pos[0][1:])
    y_i = int(split_pos[1][:-1])
    return x_i, y_i

#gcs definition
forward = 1
turn = 2
G = GraphOfConvexSets()
V = []
#add nodes 
positions = []
for x in range(0, 30, 3):
    for y in range(-15, 15, 3):
        positions.append(Position(x+1.5, y+1.5))
    
for pos in positions:
    tile = HPolyhedron(pos.A, pos.b)
    V.append(G.AddVertex(tile, pos.name))

E = []
#add edges
for pos in V:
    x_i, y_i = get_loc_from_name(pos.name())
    possible_coords = set([(x_i, y_i), 
                        (min(27, x_i + 3), y_i),
                        (min(27, x_i + 3),min(y_i + 3, 12)),
                        (min(27, x_i + 3), max(y_i - 3, -15)),
                        (x_i, min(y_i + 3, 12)),
                        (x_i, max(y_i - 3, -15))])
    for vert in V:
        x, y = get_loc_from_name(vert.name())
        if (x, y) == (x_i, y_i):
            #add stop gait
            #stop_gait = stop(...)
            #e = G.AddEdge(pos, f"({x},{y})", stop(...))
            e = G.AddEdge(pos, vert, name=f"stop_({x},{y})")
            e.AddCost(1)
            E.append(e)
        elif (x, y) in possible_coords:
            #add forward and turn edges
            forward_cost = (x-x_i)**2 + (y - y_i)**2
            turn_cost = 2*forward_cost
            for gait in [forward, turn]:
                #gait_move = gait(...)
                #e = G.AddEdge(pos, f"({x},{y})", gait)
                if gait==forward:
                    e = G.AddEdge(pos, vert, name=f"forward_({x_i},{y_i})")
                    e.AddCost(forward_cost)
                else:
                    e = G.AddEdge(pos, vert, name=f"turn_({x_i},{y_i})")
                    e.AddCost(turn_cost)
                E.append(e)

In [ ]:
#solve GCS
options = GraphOfConvexSetsOptions()
options.preprocessing = True
options.max_rounded_paths = 10
options.solver = ClarabelSolver()
options.convex_relaxation = True

print(V[5].name())
print(V[-5].name())
result = G.SolveShortestPath(V[5], V[-5], options)

In [ ]:
def extract_path(V, E, result):
    V_dict = {}
    for i, v in enumerate(V):
        V_dict[v.name()] = i
    V_adj = np.zeros((len(V), len(V)))
    yes_es = {}
    for e in E:
        if result.GetSolution(e.phi()):
            e_split = e.name().split('_')
            yes_es[e_split[1]] = e_split[0]
            u_index = V_dict[e.u().name()]
            v_index = V_dict[e.v().name()]
            V_adj[u_index, v_index] = 1
    path = ["(0,0)"]
    path_count = 0
    while path[-1] != "(27,0)":
        u_name = path[-1]
        v_index = np.where(V_adj[V_dict[u_name], :] == 1)[0][0]
        path.append(V[v_index].name())
        path_count = path_count + 1
        if path_count > 100:
            print("Abort path extraction: possible loops")
            break
    edge_types  = []
    for p in path[:-1]:
        edge_types.append(yes_es[p])
    return path, edge_types

In [ ]:
path, edge = extract_path(V, E, result)
print(path)
print(edges)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=545d651b-e7e0-4cf4-bfbe-97d6a3f869f8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>